In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
mta_df_clean = pd.read_csv('mta_df_clean_noline.csv')

In [3]:
mta_df_clean.head()

,Unnamed: 0,TURNSTILE,STATION,DATE_TIME,DATE,TIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF,T_INTERVAL,ENTRIES_RATE,EXITS_RATE
0,0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,06/05/2021,00:00:00,7581941,2591624,0.0,0.0,NaN,NaN,NaN
1,1,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,06/05/2021,04:00:00,7581950,2591625,9.0,1.0,14400.0,0.000625,0.000069
2,2,A002/R051/02-00-00,59 ST,2021-06-05 08:00:00,06/05/2021,08:00:00,7581954,2591635,4.0,10.0,14400.0,0.000278,0.000694
3,3,A002/R051/02-00-00,59 ST,2021-06-05 12:00:00,06/05/2021,12:00:00,7582009,2591655,55.0,20.0,14400.0,0.003819,0.001389
4,4,A002/R051/02-00-00,59 ST,2021-06-05 16:00:00,06/05/2021,16:00:00,7582124,2591683,115.0,28.0,14400.0,0.007986,0.001944


### In order to resample and then interpolate the data to data, we need to set the index to da DateTimeIndex:

In [4]:
# Setting the incices on DateTime:

mta_reindexed_dt = mta_df_clean.set_index(pd.DatetimeIndex(mta_df_clean['DATE_TIME']))
mta_reindexed_dt.head()

,Unnamed: 0,TURNSTILE,STATION,DATE_TIME,DATE,TIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF,T_INTERVAL,ENTRIES_RATE,EXITS_RATE
DATE_TIME,,,,,,,,,,,,,
2021-06-05 00:00:00,0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,06/05/2021,00:00:00,7581941,2591624,0.0,0.0,NaN,NaN,NaN
2021-06-05 04:00:00,1,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,06/05/2021,04:00:00,7581950,2591625,9.0,1.0,14400.0,0.000625,0.000069
2021-06-05 08:00:00,2,A002/R051/02-00-00,59 ST,2021-06-05 08:00:00,06/05/2021,08:00:00,7581954,2591635,4.0,10.0,14400.0,0.000278,0.000694
2021-06-05 12:00:00,3,A002/R051/02-00-00,59 ST,2021-06-05 12:00:00,06/05/2021,12:00:00,7582009,2591655,55.0,20.0,14400.0,0.003819,0.001389
2021-06-05 16:00:00,4,A002/R051/02-00-00,59 ST,2021-06-05 16:00:00,06/05/2021,16:00:00,7582124,2591683,115.0,28.0,14400.0,0.007986,0.001944


In [5]:
# Checking the index type:

type(mta_reindexed_dt.index)

pandas.core.indexes.datetimes.DatetimeIndex

### Now in order to make the data look uniform, we resample all the entries into 1 hour increments, and then using a linear interpolation fill in the NaN values. The resampling and then the interpolation should be done on cumulative entries and exits:

In [6]:
mta_interpolated_df = mta_reindexed_dt.groupby(['TURNSTILE'
                                                , 'STATION'])[['ENTRIES'
                                                               , 'EXITS']].resample('H').interpolate().reset_index()

mta_interpolated_df.head()

,TURNSTILE,STATION,DATE_TIME,ENTRIES,EXITS
0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,7581941.00,2591624.00
1,A002/R051/02-00-00,59 ST,2021-06-05 01:00:00,7581943.25,2591624.25
2,A002/R051/02-00-00,59 ST,2021-06-05 02:00:00,7581945.50,2591624.50
3,A002/R051/02-00-00,59 ST,2021-06-05 03:00:00,7581947.75,2591624.75
4,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,7581950.00,2591625.00


### Now we calculate the differences:

In [7]:
mta_interpolated_df[['ENTRIES_DIFF', 'EXITS_DIFF']] = mta_interpolated_df.groupby(['TURNSTILE'])[['ENTRIES'
                                                                                                  , 'EXITS']].diff()

mta_interpolated_df.head()

,TURNSTILE,STATION,DATE_TIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,7581941.00,2591624.00,NaN,NaN
1,A002/R051/02-00-00,59 ST,2021-06-05 01:00:00,7581943.25,2591624.25,2.25,0.25
2,A002/R051/02-00-00,59 ST,2021-06-05 02:00:00,7581945.50,2591624.50,2.25,0.25
3,A002/R051/02-00-00,59 ST,2021-06-05 03:00:00,7581947.75,2591624.75,2.25,0.25
4,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,7581950.00,2591625.00,2.25,0.25


### In the initial data cleaning process, when dealing with negatives, outliers and off counters, we corrected and dropped the DIFF columns and left the cumulative columns intact, now as a result of resampling and interpolation, our new calculated DIFFS are going to include negatives, outliers and off counters. So we have to address and clean those:

In [8]:
# Correcting negative values:

mta_interpolated_df[['ENTRIES_DIFF', 'EXITS_DIFF']] = mta_interpolated_df[['ENTRIES_DIFF', 'EXITS_DIFF']].abs()

In [9]:
mta_interpolated_df[mta_interpolated_df['ENTRIES_DIFF'] < 0]

,TURNSTILE,STATION,DATE_TIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF


In [10]:
# Dropping of the entry outliers (1200 maximum hourly entry):

mta_interpolated_df.loc[mta_interpolated_df.ENTRIES_DIFF > 1200, 'ENTRIES_DIFF'] = np.nan

In [11]:
# Dropping of the exit outliers(1800 maximum hourly entry):

mta_interpolated_df.loc[mta_interpolated_df.EXITS_DIFF > 1800, 'EXITS_DIFF'] = np.nan

### IMPORTANT NOTATION: A backward shift in the structure of the data can be detected upon inspection. This is due to unsampling and extrapolating. In order to avoid assigning wrong numbers to timestamps when we resample back to 4H increments, we have to shift the data for each turnstile 3 rows down:  

In [12]:
mta_interpolated_df.head()

,TURNSTILE,STATION,DATE_TIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,7581941.00,2591624.00,NaN,NaN
1,A002/R051/02-00-00,59 ST,2021-06-05 01:00:00,7581943.25,2591624.25,2.25,0.25
2,A002/R051/02-00-00,59 ST,2021-06-05 02:00:00,7581945.50,2591624.50,2.25,0.25
3,A002/R051/02-00-00,59 ST,2021-06-05 03:00:00,7581947.75,2591624.75,2.25,0.25
4,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,7581950.00,2591625.00,2.25,0.25


In [13]:
# Shifting the diffs down 3 rows for each individual turnstile:

mta_interpolated_df[['ENTRIES_DIFF'
                     , 'EXITS_DIFF']] = mta_interpolated_df.groupby(['TURNSTILE'])[['ENTRIES_DIFF'
                                                                                    , 'EXITS_DIFF']].shift(periods=3)
mta_interpolated_df.head()

,TURNSTILE,STATION,DATE_TIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,7581941.00,2591624.00,NaN,NaN
1,A002/R051/02-00-00,59 ST,2021-06-05 01:00:00,7581943.25,2591624.25,NaN,NaN
2,A002/R051/02-00-00,59 ST,2021-06-05 02:00:00,7581945.50,2591624.50,NaN,NaN
3,A002/R051/02-00-00,59 ST,2021-06-05 03:00:00,7581947.75,2591624.75,NaN,NaN
4,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,7581950.00,2591625.00,2.25,0.25


### Off counters will be dealt with after reasmpling into 4H.

### Now once again we need to reindex the df to reample back to 4H:

In [14]:
# Resetting indices to DateTimeIndex:

mta_interpolated_df.set_index(pd.DatetimeIndex(mta_interpolated_df['DATE_TIME']), inplace=True)
mta_interpolated_df.head()

,TURNSTILE,STATION,DATE_TIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
DATE_TIME,,,,,,,
2021-06-05 00:00:00,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,7581941.00,2591624.00,NaN,NaN
2021-06-05 01:00:00,A002/R051/02-00-00,59 ST,2021-06-05 01:00:00,7581943.25,2591624.25,NaN,NaN
2021-06-05 02:00:00,A002/R051/02-00-00,59 ST,2021-06-05 02:00:00,7581945.50,2591624.50,NaN,NaN
2021-06-05 03:00:00,A002/R051/02-00-00,59 ST,2021-06-05 03:00:00,7581947.75,2591624.75,NaN,NaN
2021-06-05 04:00:00,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,7581950.00,2591625.00,2.25,0.25


### And we finally resample back to 4 hours increments: 

In [15]:
# Resampling back to 4H:

mta_resampled_df = mta_interpolated_df.groupby(['TURNSTILE'
                                                , 'STATION'])[['ENTRIES_DIFF'
                                                               , 'EXITS_DIFF']].resample('4H').sum().reset_index()

In [16]:
mta_resampled_df.head()

,TURNSTILE,STATION,DATE_TIME,ENTRIES_DIFF,EXITS_DIFF
0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,0.0,0.0
1,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,9.0,1.0
2,A002/R051/02-00-00,59 ST,2021-06-05 08:00:00,4.0,10.0
3,A002/R051/02-00-00,59 ST,2021-06-05 12:00:00,55.0,20.0
4,A002/R051/02-00-00,59 ST,2021-06-05 16:00:00,115.0,28.0


In [17]:
mta_resampled_df.describe()

,ENTRIES_DIFF,EXITS_DIFF
count,2.731541e+06,2.731541e+06
mean,6.260759e+01,6.816778e+01
std,1.296165e+02,1.110898e+02
min,0.000000e+00,0.000000e+00
25%,2.079916e+00,4.750000e+00
50%,2.500000e+01,2.900000e+01
75%,8.100000e+01,8.700000e+01
max,4.786166e+03,4.171500e+03


### Finally we remove the off counters:

In [18]:
off_entry_counters = pd.read_csv('off_entry_counters.csv')
off_exit_counters = pd.read_csv('off_exit_counters.csv')

for turnstile in off_entry_counters['TURNSTILE']:
    mta_resampled_df.loc[mta_resampled_df.TURNSTILE == turnstile, 'ENTRIES_DIFF'] = np.nan

for turnstile in off_exit_counters['TURNSTILE']:
    mta_resampled_df.loc[mta_resampled_df.TURNSTILE == turnstile, 'EXITS_DIFF'] = np.nan

In [19]:
mta_resampled_df.head()

,TURNSTILE,STATION,DATE_TIME,ENTRIES_DIFF,EXITS_DIFF
0,A002/R051/02-00-00,59 ST,2021-06-05 00:00:00,0.0,0.0
1,A002/R051/02-00-00,59 ST,2021-06-05 04:00:00,9.0,1.0
2,A002/R051/02-00-00,59 ST,2021-06-05 08:00:00,4.0,10.0
3,A002/R051/02-00-00,59 ST,2021-06-05 12:00:00,55.0,20.0
4,A002/R051/02-00-00,59 ST,2021-06-05 16:00:00,115.0,28.0


In [20]:
mta_resampled_df.describe()

,ENTRIES_DIFF,EXITS_DIFF
count,2.592311e+06,2.585912e+06
mean,6.546652e+01,7.198387e+01
std,1.313063e+02,1.129643e+02
min,0.000000e+00,0.000000e+00
25%,4.250000e+00,7.750000e+00
50%,2.825000e+01,3.300000e+01
75%,8.550000e+01,9.200000e+01
max,4.786166e+03,4.171500e+03


In [21]:
# Writing the resampled df into a CSV file to use for further analysis:

mta_resampled_df.to_csv('mta_resampled_df.csv')